In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
base_url = 'https://www.numbeo.com/cost-of-living/historical-data-city-selector'

In [ ]:
page = requests.get(base_url)
numbeo_city_soup = BeautifulSoup(page.content, "html.parser")
results = numbeo_city_soup.find('table', class_='related_links')
print(results())

In [6]:
list_cities = results.find_all('a')
list_cities[0]

<a href="https://www.numbeo.com/cost-of-living/city-history/in/Aachen">Aachen, Germany</a>

In [7]:
len(list_cities)

964

In [8]:
city_dict = lambda x: {'City': x.text.split(",")[0], 'Country':x.text.split(",")[1], 'Url':x["href"]}
city_pages = [city_dict(city) for city in list_cities]

In [10]:
city_pages[0]

{'City': 'Aachen',
 'Country': ' Germany',
 'Url': 'https://www.numbeo.com/cost-of-living/city-history/in/Aachen'}

In [11]:
df = pd.DataFrame(city_pages)
df.head()

,City,Country,Url
0,Aachen,Germany,https://www.numbeo.com/cost-of-living/city-his...
1,Aalborg,Denmark,https://www.numbeo.com/cost-of-living/city-his...
2,Aarhus,Denmark,https://www.numbeo.com/cost-of-living/city-his...
3,Abbotsford,Canada,https://www.numbeo.com/cost-of-living/city-his...
4,Aberdeen,United Kingdom,https://www.numbeo.com/cost-of-living/city-his...


In [13]:
df['City'] == 'Abbotsford'

0      False
1      False
2      False
3       True
4      False
       ...  
959    False
960    False
961    False
962    False
963    False
Name: City, Length: 964, dtype: bool

In [24]:
df.loc[df['City'] == 'Abbotsford']

,City,Country,Url
3,Abbotsford,Canada,https://www.numbeo.com/cost-of-living/city-his...


In [20]:
df.loc[df['City'] == 'Abbotsford'].index[0]

3

In [25]:
df.iloc[df.loc[df['City'] == 'Abbotsford'].index[0]]["Url"]

'https://www.numbeo.com/cost-of-living/city-history/in/Abbotsford'

In [26]:
page = requests.get(df.iloc[df.loc[df['City'] == 'Abbotsford'].index[0]]["Url"])

In [27]:
page

<Response [200]>

In [29]:
abbotsford = BeautifulSoup(page.content, "html.parser")

In [ ]:
abbotsford

In [31]:
inner_width = abbotsford.find_all('div', class_='innerWidth')

In [36]:
len(inner_width)

3

In [ ]:
inner_width[2]

In [41]:
def get_tables(city):
    page = requests.get(df.iloc[df.loc[df['City'] == city].index[0]]["Url"])
    one_city_soup = BeautifulSoup(page.content, "html.parser")
    inner_width = one_city_soup.find_all('div', class_='innerWidth')
    results = inner_width[2].find_all('table')
    return results

In [42]:
cape_town_data = get_tables("Cape Town")
len(cape_town_data) 

14

In [44]:
cape_town_data[0]

<table class="stripe row-border order-column compact" id="tier_1">
<thead>
<tr>
<th><div class="font_in_table_headers">Year</div></th><th><div class="font_in_table_headers">Meal, Inexpensive Restaurant</div></th><th><div class="font_in_table_headers">Meal for 2 People, <br/>Mid-range Restaurant, Three-course</div></th><th><div class="font_in_table_headers">McMeal at McDonalds <br/>(or Equivalent Combo Meal)</div></th></tr>
</thead>
<tbody>
<tr>
<td style="text-align: right">2023</td>
<td style="white-space: nowrap; text-align: right">170.00</td>
<td style="white-space: nowrap; text-align: right">700.00</td>
<td style="white-space: nowrap; text-align: right">80.00</td>
</tr>
<tr>
<td style="text-align: right">2022</td>
<td style="white-space: nowrap; text-align: right">150.00</td>
<td style="white-space: nowrap; text-align: right">600.00</td>
<td style="white-space: nowrap; text-align: right">70.00</td>
</tr>
<tr>
<td style="text-align: right">2021</td>
<td style="white-space: nowrap; t

In [43]:
nairobi_data = get_tables("Nairobi")
len(nairobi_data)

14

In [45]:
from io import StringIO

In [46]:
reader_converter = lambda x: pd.DataFrame(pd.read_html(StringIO(str(x)))[0])

cape_df_list = [reader_converter(table) for table in cape_town_data]
nairobi_df_list =[reader_converter(table) for table in nairobi_data]

In [48]:
cape_df_list[13]

,Year,1 Pair of Jeans (Levis 501 Or Similar),"1 Summer Dress in a Chain Store (Zara, H&M, ...)",1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes
0,2023,822.46,541.04,1548.28,1367.35
1,2022,824.94,562.93,1545.66,1299.52
2,2021,712.03,533.33,1327.75,1327.16
3,2020,755.44,506.53,1333.99,1293.25
4,2019,788.30,502.96,1382.25,1253.38
5,2018,758.63,456.52,1206.04,1164.58
6,2017,693.21,467.50,1114.10,993.55
7,2016,721.54,501.47,1071.40,1186.17
8,2015,667.00,488.85,943.18,843.52
9,2014,643.25,394.22,924.92,766.58


In [40]:
nairobi_df_list[0]

,Year,Domestic Beer (0.5 liter draught),Imported Beer (0.33 liter bottle),Coke/Pepsi (0.33 liter bottle),Water (0.33 liter bottle),Cappuccino (regular)
0,2023,299.58,400.0,70.42,58.00,321.10
1,2022,250.00,350.0,69.91,60.26,282.40
2,2021,250.00,350.0,52.94,42.81,247.33
3,2020,250.00,300.0,53.33,56.58,305.00
4,2019,300.00,400.0,69.02,69.81,282.44
5,2018,201.56,275.0,47.76,44.25,230.16
6,2017,200.00,250.0,57.69,49.00,237.27
7,2016,200.00,300.0,55.75,47.27,250.71
8,2015,200.00,300.0,71.33,55.39,228.00
9,2014,200.00,280.0,63.36,50.76,234.23


In [34]:
import os

def save_df(df_list, city): 
    titles = ['eat-out-meal', 'beverage', 'basic-grocery', 'fresh-produce', 'deli', 'rental', 'property',
              'salary', 'transport-daily', 'transport-monthly', 'mortgage', 'utilities', 'entertainment', 'clothing']
    # Create directory if it doesn't exist
    directory = f'data/{city}'
    if not os.path.exists(directory):
        os.makedirs(directory)
    for index, frame in enumerate(df_list):
        # Ensure the title index is within bounds
        if index < len(titles):
            # Save DataFrame to CSV
            frame.to_csv(f'{directory}/{titles[index]}.csv', sep=',', index=False, encoding='utf-8')


In [35]:
save_df(cape_df_list, 'cape-town')

In [36]:
save_df(nairobi_df_list, 'nairobi')